<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **SpaceX  Falcon 9 First Stage Landing Prediction**


# Part 7: SpaceX Launches - Records Dashboard Dash App

**Short description:**  
This notebook contains a Dash application that visualizes SpaceX launch records. It provides interactive controls to filter by launch site and payload range, and displays a pie chart of successes and a scatter chart showing the correlation between payload mass and launch success.

**Objectives**
- Load SpaceX launch data into a DataFrame for visualization.
- Build an interactive Dash dashboard with dropdown and range-slider controls.
- Visualize total successful launches per site (pie chart) and payload vs. success correlation (scatter chart).
- Demonstrate callback wiring in Dash to link controls to visualizations.

**Notice about documentation:**  
The original notebook submission (course assignment) was kept intact. I have **only modified documentation (comments, headings, markdown)** and made **minimal, necessary corrections** to ensure the notebook runs without errors. All rights related to the lab/workshop design and original exercise belong exclusively to **IBM Corporation**. This notebook includes additional documentation for clarity, but the intellectual property of the original exercise is retained by IBM.

---

## Table of contents

1. Dependencies & execution instructions  
2. Data loading & quick inspection  
3. App layout (controls, charts)  
4. Callbacks (pie chart and scatter chart)  
5. Running the app & deployment notes


## 1) Dependencies & execution instructions

This section installs and imports required Python packages.  

**Recommended local execution steps:**

1. Create and activate a Python virtual environment:
   - `python -m venv venv`
   - `source venv/bin/activate` (macOS / Linux) or `venv\Scripts\activate` (Windows)
2. Install dependencies:
   - `pip install -r requirements.txt`
3. Run the app:
   * Run `7_Interactive_Visual_Analytics_Plotly.py`
   * Run the notebook. Ensure the environment supports Dash in notebooks (or run as script).

**Note:** The code uses `dash_html_components` and `dash_core_components` import style — keep packages compatible with that API.

In [ ]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px


## 2) Data loading & quick inspection

Load the CSV `spacex_launch_dash.csv` into a pandas DataFrame and determine `min_payload` and `max_payload` used by the payload-range slider.


In [ ]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

## 3) App layout — controls and chart placeholders

This section defines the Dash layout:
- Dropdown `site-dropdown` to select All sites or a specific launch site.
- Pie chart placeholder (`success-pie-chart`) that shows total successes (or success vs failure for a single site).
- RangeSlider `payload-slider` to select payload range.
- Scatter chart placeholder (`success-payload-scatter-chart`) that shows correlation between payload and success colored by Booster category.

In [ ]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # Dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                ],
                                                value = 'ALL',
                                                placeholder = 'Select a Launch Site here',
                                                searchable = True),
                                html.Br(),

                                # Pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # Slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                marks={i: '{}'.format(i) for i in range(0, 10001, 1000)},
                                                value=[min_payload,max_payload]),

                                # Scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

## 4) Callbacks — interactive chart updates

Two callbacks:
- Updates the pie chart when `site-dropdown` changes.
- Updates the scatter chart when `site-dropdown` or `payload-slider` change.

Behavior:
- Pie chart: for 'ALL' shows total successes per site, else shows success vs failure counts for selected site.
- Scatter chart: filters points by payload range and selected site; x-axis is payload mass, y-axis is class (0/1) and colored by Booster Version Category.

In [ ]:
# Callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(launch_site):
    filtered_df = spacex_df[spacex_df['Launch Site'] == launch_site]
    if launch_site == 'ALL':
        fig = px.pie(spacex_df.groupby('Launch Site')['class'].sum().reset_index(),
                    values='class', 
                    names='Launch Site', 
                    title='Total Success Launches by Site')
    else:
        filtered_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        fig = px.pie(filtered_df, values='class count', names='class', 
                     title=f"Total Success Launches for site {launch_site}")
    return(fig)

# Callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
               [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])

def get_payload_chart(launch_site, payload_mass):
    if launch_site == 'ALL':
        fig = px.scatter(spacex_df[spacex_df['Payload Mass (kg)'].between(payload_mass[0], payload_mass[1])], 
                x="Payload Mass (kg)",
                y="class",
                color="Booster Version Category",
                hover_data=['Launch Site'],
                title='Correlation Between Payload and Success for All Sites')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == launch_site]
        fig = px.scatter(filtered_df[filtered_df['Payload Mass (kg)'].between(payload_mass[0], payload_mass[1])], 
                x="Payload Mass (kg)",
                y="class",
                color="Booster Version Category",
                hover_data=['Launch Site'],
                title='Correlation Between Payload and Success for Site {}'.format(launch_site))
    return(fig)

## 5) Running the app & deployment notes

- Run the app locally with the final `if __name__ == '__main__': app.run_server(port=8054)` block.
- For deployment, use `7_Interactive_Visual_Analytics_Plotly.py`, ensure required packages are listed in `requirements.txt`, and configure the hosting environment (e.g., use Gunicorn, set proper host/port, turn off debug).
- If the dataset is not available locally, generate `spacex_launch_dash.csv` with the URL and ensure the app can access it in the deployment environment.

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run(port=8054)